Henneken, Edwin

May 3, 2022
	

Hi Ryan

I created a sample script that queries the ADS API. All you need to do is replace TOKEN by your ADS API token in this line

APItoken = 'TOKEN'

You can find your token here (when you're logged in to your ADS account): https://ui.adsabs.harvard.edu/user/settings/token

The file takes an input ASCII file (I attached an example) which has on each line a file name and an ADS query, separated by a tab. The file name will be the name of the TSV file that will hold the results of the query; a date string will be included in the name of the TSV file, so if you execute this script, say, once a month, previous output won't be overwritten. In its current version, the TSV file has 6 columns:

1: bibcode
2: DOI
3: first author
4: article title
5: refereed status ("1" is refereed, "0" is nonrefereed)
6: open access status ("1" is open access, "0" is not open access)

The script is executed like

  python doADS_API_Query.py queries.txt

I hope this is useful.

--Edwin



Questions
 - use of urllib2 (deprecated for Python3?)
 - ...
 
 

In [23]:
# Import standard Python libraries
# import urllib2
# import urllib
import urllib3
import json
import sys
import math
import csv
from datetime import datetime

import pandas as pd


'%s?%s'

In [37]:
## Helper functions
# Execute a search query
def do_query(URL, params):
    qparams = urllib.parse.urlencode(params)
    http = urllib3.PoolManager()
    req = http.request("%s?%s".format(URL, qparams))
    # and add the correct header information
    req.add_header('Content-type', 'application/json')
    req.add_header('Accept', 'text/plain')
    req.add_header('Authorization', 'Bearer %s' % APItoken)
    # do the actual request
    resp = urllib3.urlopen(req)
    # and retrieve the data to work with
    data = json.load(resp)
    return data
# Get records from Solr
def get_records(token, query_string, return_fields):
    start = 0
    results = []
    params = {
        'q':query_string,
        'fl': return_fields,
        'rows': rows,
        'start': start
    }
    data = do_query(QUERY_URL, params)
    try:
        results = data['response']['docs']
    except:
        raise Exception('Solr returned unexpected data!')
    num_documents = int(data['response']['numFound'])
    num_paginates = int(math.ceil((num_documents) / (1.0*rows))) - 1
    start += rows
    for i in range(num_paginates):
        params['start'] = start
        data = do_query(QUERY_URL, params)
        try:
            results += data['response']['docs']
        except:
            raise Exception('Solr returned unexpected data!')
        start += rows
    return results



In [14]:
query_file = '/Users/ryanmcgranaghan/Documents/Helio_ECIP/dev/Helio-KNOW/ADS_enrichment/queries.txt'

queries = open(query_file).read().strip().split('\n')
queries

['# each line: two entries separated by tab',
 '# first column: name of output file',
 '# second column: ADS query',
 'active_solar_chromosphere\tkeyword_schema:UAT keyword:1980']

In [38]:
## Input parameters
# Bumblebee API token
APItoken = 'Qe9f4qU8ITbD0rP7rIRNvlgxZ5vYfahY8PW05en0'


# Address of API
API_URL = 'https://api.adsabs.harvard.edu/v1'
QUERY_URL = "%s/search/query" % API_URL


# Query parameters
# The number of records to be returned in Solr query
rows = 300


# What data do we need back from Solr
fields = "bibcode,doi,first_author_norm,title,property"


# date string for output file
dstring = datetime.today().strftime('%Y%m%d')


# ## Get the query for which we want the data
# try:
#     query_file = sys.argv[1]
# except:
#     sys.exit('Please provide name for file with queries as argument...')

query_file = 'queries.txt'
# query_input = 'active_solar_chromosphere    keyword_schema:UAT keyword:1980' #RMM: manually input a sample query here
    

### Main part of script
# get the entries in the input file
try:
    queries = open(query_file).read().strip().split('\n')
except Exception:
    sys.exit('Failed to get queries from file')

    
# Now execute the queries
for entry in queries:
    # ignore comment lines
    if entry.startswith('#'):
        continue
    # get file name and query
    fname, query = entry.split('\t')
    # retrieve the records found by the query
    try:
        pubdata = get_records(APItoken, query, fields)
    except Exception:
        sys.exit('Failed to get results for query provided')
    # determine output file
    ofile = "{0}_{1}.tsv".format(fname, dstring)
    # save some data in the records retrieved to the TSV file
    with open(ofile, 'w') as out_file:
        tsv_writer = csv.writer(out_file, delimiter='\t')
        for entry in pubdata:
            properties = entry['property']
            refstatus = 1
            if 'REFEREED' not in properties:
                refstatus = 0
            openaccess = 1
            if 'OPENACCESS' not in properties:
                openaccess = 0
            row = []
            row.append(entry['bibcode'])
            row.append(entry['doi'][0])
            row.append(entry['first_author_norm'])
            row.append(entry['title'][0].encode('utf-8'))
            row.append(refstatus)
            row.append(openaccess)
            tsv_writer.writerow(row)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "<ipython-input-38-cd9c1d86c06d>", line 51, in <module>
    pubdata = get_records(APItoken, query, fields)
  File "<ipython-input-37-057a13c48be5>", line 26, in get_records
    data = do_query(QUERY_URL, params)
  File "<ipython-input-37-057a13c48be5>", line 6, in do_query
    req = http.request("%s?%s".format(URL, qparams))
TypeError: request() missing 1 required positional argument: 'url'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/ryanmcgranaghan/opt/anaconda3/envs/pythontf/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-38-cd9c1d86c06d>", line 53, in <module>
    sys.exit('Failed to get results for query provided')
SystemExit: Failed to get results for query provided

During handling of the above exception, another exception occurred:

Traceback (most r

TypeError: object of type 'NoneType' has no len()

In [ ]:
# Read the output file
query_outputs = open(out_file)

In [20]:
for entry in queries:
    # ignore comment lines
    if entry.startswith('#'):
        continue
    # get file name and query
    fname, query = entry.split('\t')
query

'keyword_schema:UAT keyword:1980'